In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu" )
device

device(type='cpu')

In [3]:
transfrom = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])


In [4]:
train_dataset = torchvision.datasets.CIFAR10(root='data', train=True,download=True,transform=transfrom)
test_dataset = torchvision.datasets.CIFAR10(root='data', train=False,download=True,transform=transfrom)

In [5]:
len(train_dataset), len(test_dataset)

(50000, 10000)

In [6]:
batch_size = 300

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size,shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size,shuffle=False)

In [7]:
for i ,(images,labels) in enumerate(train_loader):
    print(images.shape)
    print(labels.shape)
    break

torch.Size([300, 3, 32, 32])
torch.Size([300])


In [8]:
labels[:8]

tensor([1, 5, 1, 7, 0, 5, 8, 2])

In [9]:
classes =("plane",'car','bird','cat','deer','dog','frog','horse','ship','truck')

In [10]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3,32,kernel_size=(3,3),padding='same'), #output(32,32x32)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=(2,2)), # output(32,16x16)
            nn.Conv2d(32, 64,kernel_size=(3,3)), # output(64, 14x14)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=(2,2)),# output(64,7x7)
        )
        self.fc_layers = nn.Sequential(
                nn.Flatten(),
                nn.Linear(64*7*7,600),
                nn.ReLU(),
                nn.Linear(600,120),
                nn.ReLU(),
                nn.Linear(120,10)
        )
    def forward(self,x):
        x = self.network(x)
        x = self.fc_layers(x)
        return x

In [20]:
# Training function
def train_model(model, criterion, optimizer, train_loader, val_loader, epochs=5):
    train_losses, val_losses, val_accuracy = [], [], []
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_losses.append(train_loss / len(train_loader))
        
        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
        val_losses.append(val_loss / len(val_loader))
        val_accuracy.append(correct/total)
        
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}, Val Accuracy:  {100 * val_accuracy[-1]:.2f}%")
        
    return train_losses, val_losses, val_accuracy

In [21]:
# Hyperparameters
num_epochs = 5
learning_rate = 0.001

model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train without augmentation
train_losses, val_losses, val_accuracy = train_model(model, criterion, optimizer, train_loader, test_loader, epochs=5)

Epoch 1/5, Train Loss: 1.5338, Val Loss: 1.3183, Val Accuracy:  52.84%
Epoch 2/5, Train Loss: 1.1630, Val Loss: 1.0910, Val Accuracy:  60.42%
Epoch 3/5, Train Loss: 0.9829, Val Loss: 0.9660, Val Accuracy:  66.04%
Epoch 4/5, Train Loss: 0.8448, Val Loss: 0.8947, Val Accuracy:  68.36%
Epoch 5/5, Train Loss: 0.7405, Val Loss: 0.8709, Val Accuracy:  69.56%


With DATA Augmentation

In [22]:
# import torchvision.transforms as transforms

transform_with_aug = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dataset_aug = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_with_aug)
train_loader_aug = DataLoader(train_dataset_aug, batch_size=32, shuffle=True)

model_aug = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_aug.parameters(), lr=learning_rate)

train_losses, val_losses, val_accuracy = train_model(model_aug, criterion, optimizer, train_loader_aug, test_loader, epochs=5)

Epoch 1/5, Train Loss: 1.4338, Val Loss: 1.1187, Val Accuracy:  60.31%
Epoch 2/5, Train Loss: 1.0664, Val Loss: 0.9169, Val Accuracy:  67.91%
Epoch 3/5, Train Loss: 0.9375, Val Loss: 0.8677, Val Accuracy:  70.04%
Epoch 4/5, Train Loss: 0.8489, Val Loss: 0.8396, Val Accuracy:  70.84%
Epoch 5/5, Train Loss: 0.7893, Val Loss: 0.7791, Val Accuracy:  73.19%
